Necessary imports

In [3]:
import os.path as p
import pandas as pd
import nltk
import spacy as s

from definitions import *
from dataset_helper_functions import *

Global switches.

In [ ]:
analyze_pos = True

Spacy configuration

In [47]:
spacy = s.load('en_core_web_lg') # en_core_web_trf for accuracy
stopwords = (set(nltk.corpus.stopwords.words('english')))

The following code only needs to be run once at the start.

In [5]:
# combine_debates()
# create_validation_subset()
# sample_development_set()

In [91]:
train_path = p.join(POLIT_DATA_DIR_PATH, 'train')
dev = pd.read_csv(p.join(train_path, 'dev.tsv'), sep='\t', index_col=False)

Apply spacy to content.

In [92]:
dev['spacy'] = dev['content'].apply(lambda x: spacy(x)).values
# dev['spacy_no_stop'] = dev['spacy'].apply(lambda x: [t for t in x if t not in stopwords])     

POS tags analysis
- `pos_` represents coarse-grained POS tag described [here](https://universaldependencies.org/u/pos/)
- `tag_` represents fine-grained POS tag

Initialize pos and tag dataframes.

In [132]:
if analyze_pos:
    pos_df = pd.DataFrame(
        index=pd.MultiIndex.from_product(
            [
                ['worthy', 'unworthy'],
                ['stop', 'nostop'],
            ]
        ),
    )
    tag_df = pd.DataFrame(
        index=pd.MultiIndex.from_product(
            [
                ['worthy', 'unworthy'],
                ['stop', 'nostop'],
            ]
        ),
    )

Fill pos and tag dataframes.

In [133]:
if analyze_pos:
    for _, sent, label in dev[['spacy', 'label']].itertuples():
        worthiness = 'worthy' if label == 1 else 'unworthy'

        for t in sent:
            # ignore punctuation
            if t.is_punct:
                continue

            stop = 'stop' if t.is_stop else 'nostop'

            if t.pos_ in pos_df.columns:
                pos_df.loc[(worthiness, stop), t.pos_] += 1
            else:
                pos_df.loc[:, t.pos_] = 0
                pos_df.loc[(worthiness, stop), t.pos_] = 1

            if t.tag_ in tag_df.columns:
                tag_df.loc[(worthiness, stop), t.tag_] += 1
            else:
                tag_df.loc[:, t.tag_] = 0
                tag_df.loc[(worthiness, stop), t.tag_] = 1

    

In [135]:
print(pos_df)
pos_df.loc['worthy', 'stop'] = pos_df.loc['worthy', 'stop'] + pos_df.loc['worthy', 'nostop']
print(pos_df)

                 PROPN   AUX   DET  ADJ  NOUN  ADV   ADP  PRON  VERB  PART  \
worthy   stop        0    49    64    3     3   20    79    75    24    20   
         nostop     45     1     0   40   130    8     1     0    72     0   
unworthy stop        6  1107  1371  174    42  672  1238  2076   740   594   
         nostop    763     7     0  659  2034  214    26    18  1629     0   

                 CCONJ  NUM  SCONJ  INTJ  SYM  X  SPACE  PUNCT  
worthy   stop       27    7     17     0    0  0      0      0  
         nostop      0    7      0     0    3  0      0      0  
unworthy stop      589   61    259    38    0  2      0      6  
         nostop      0   95      2    48   19  2      4      0  
                 PROPN   AUX   DET  ADJ  NOUN  ADV   ADP  PRON  VERB  PART  \
worthy   stop       45    50    64   43   133   28    80    75    96    20   
         nostop     45     1     0   40   130    8     1     0    72     0   
unworthy stop        6  1107  1371  174    42  672

- POS feature could be represented by one hot encoding based on whether sentence contains given POS tag or not
- or can be represented as a number concatenated from counts of POS tags in a sentence including zeroes for POS not present

In [ ]:
# temp = dev.loc[0, 'content']
# x = spacy(temp)
# temp = dev[['id', 'content']]
# temp['spacied'] = temp['content'].apply(lambda x: spacy(x))
# # temp = ['tag_ | pos_ | dep_ | lemma_ | norm_']
# # for t in x:
# #     temp.append(f'{t.tag_} | {t.pos_} | {t.dep_} | {t.lemma_} | {t.norm_}')
# print(temp.loc[0, 'spacied'][0].tag_)
# [(i.text, i.pos_, i.) for i in x]